In [ ]:
!nvidia-smi

Mon Jan 24 01:56:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import cv2
import math
import keras
import numpy as np
from skimage import io
from sys import getsizeof
from tensorflow import keras
from sklearn.metrics import f1_score
from skimage.transform import rescale
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
seed = 7
np.random.seed(seed)

data_path = '/content/drive/MyDrive/Newcastle University/Machine Learning/lesions'

def load_paths(path):
  
  image_paths = []
  y = []
  
  for label, elem in enumerate(os.listdir(path)):

    print('class: ' + elem + ' | label: ' + str(label))
    path1 = path + '/' + elem
    images = os.listdir(path1)
    for im in images:
      image_paths.append(path1 + '/' + im)
      y.append(str(label))
  
  # shuffle
  c = list(zip(image_paths,y))
  np.random.shuffle(c)
  image_paths,y = zip(*c)
  
  return image_paths, keras.utils.to_categorical(y, num_classes=7)

image_paths, labels = load_paths(data_path)

x_train, x_test, y_train, y_test = train_test_split(image_paths, labels, test_size = 0.2, random_state = 42)

In [ ]:
# Training data (non-augmented)
train_path1 = '/content/drive/MyDrive/Newcastle University/Machine Learning/Dataset/train'
# Training data (augmented)
train_path2 = '/content/drive/MyDrive/Newcastle University/Machine Learning/Dataset/output'
# Testing data
valid_path = '/content/drive/MyDrive/Newcastle University/Machine Learning/Dataset/validation'

In [ ]:
classes = sorted(os.listdir(train_path2))

In [ ]:
train_list = []
train_labels = []

#elem_list1 = os.listdir(path_train)

for label, elem in enumerate(classes):
  
  path1 = train_path2 + '/' + str(elem)
  images = os.listdir(path1)
  
  for im in images:
      
      # takes the whole path of the images
      # removes the need to mention the dataset path for the generator
      train_list.append(path1 + '/' + str(im))
      train_labels.append(label)

# shuffling the image list
c = list(zip(train_list, train_labels))
np.random.shuffle(c)
train_list, train_labels = zip(*c)

# one hot encoding
train_labels = keras.utils.to_categorical(train_labels, num_classes=7)

In [ ]:
test_list = []
test_labels = []

#elem_list1 = os.listdir(path_train)

for label, elem in enumerate(classes):
  
  path1 = valid_path + '/' + str(elem)
  images = os.listdir(path1)
  
  for im in images:
      
      # takes the whole path of the images
      # removes the need to mention the dataset path for the generator
      test_list.append(path1 + '/' + str(im))
      test_labels.append(label)

test_labels = keras.utils.to_categorical(test_labels, num_classes=7)

In [ ]:
train_labels.shape

(20000, 7)

In [ ]:
test_labels.shape

(2005, 7)

# Generator for data loading while training the model
def im_datagen(image_list, label_list, batch_size = 32, shuffle = False):
    while True:
        # shuffle the data
        if shuffle == True:
            c = list(zip(image_list, label_list))
            np.random.shuffle(c)
            image_list, label_list = zip(*c)
    
        total_elements = len(label_list)
    
        for i in range(0, total_elements, batch_size):
          
          images = [cv2.resize(cv2.imread(x), (224,224)) for x in image_list[i:i+batch_size]]
          images = np.asarray(images)
          images = images/255.
          
          yield images, keras.utils.to_categorical(label_list[i:i+batch_size], num_classes = 7)

batch_size = 50

STEP_SIZE_TRAIN = len(train_labels)//batch_size
STEP_SIZE_VALID = len(test_labels)//batch_size

print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

train_gen = im_datagen(train_list, train_labels, batch_size = batch_size, shuffle = True)

test_gen = im_datagen(test_list, test_labels, batch_size = batch_size, shuffle = False)

x_test.[17]

y_test[17]

Counter(y_test).keys() # equals to list(set(words))

Counter(y_test).values() # counts the elements' frequency

In [ ]:
class LeisonSequence(keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size = 32):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        images = []

        for file_name in batch_x:
          images.append(cv2.resize(cv2.imread(file_name), (224, 224)))

        return (np.array(images)/255), np.array(batch_y)

from matplotlib import pyplot as plt
test = []

for file_name in x_test[0:5]:
  test.append(rescale(io.imread(file_name), (0.5, 0.5, 1)))
  #print(file_name)

test = np.array(test)
plt.imshow(test[3])

In [ ]:
train_img_gen = LeisonSequence(train_list, train_labels, 50)
val_img_gen = LeisonSequence(test_list, test_labels, 50)

In [ ]:
#base_model = keras.applications.ResNet50(
model = keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(224, 224, 3),
    classes=7,
    pooling=None
)

In [ ]:
#base_model.summary()
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

inputs = keras.Input(shape=(224, 224, 3))
last_layer = base_model.get_layer('global_average_pooling2d')

print ('last layer output shape:', last_layer.output_shape)

bm_out = last_layer.output
#x = base_model(inputs, training=False)
x = keras.layers.Dense(128, activation='relu')(bm_out)
#x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(7, activation='softmax')(x)
#model = keras.Model(inputs, outputs)
model = keras.Model(base_model.input, outputs)

model.summary()

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0002,
    decay_steps=100,
    decay_rate=0.9)
sgd = keras.optimizers.SGD(learning_rate=0.0002)
adam = keras.optimizers.Adam(learning_rate=lr_schedule, amsgrad=True)
nadam = keras.optimizers.Nadam(learning_rate=0.0002)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=nadam ,metrics=['accuracy'])

In [ ]:
model.load_weights('/content/drive/MyDrive/Newcastle University/Machine Learning/Models/resnet50_aug.h5')

In [ ]:
cp = keras.callbacks.ModelCheckpoint(filepath = '/content/drive/MyDrive/Newcastle University/Machine Learning/Models/resnet50_aug.h5', verbose = 1, save_best_only = True, monitor='val_accuracy')

In [ ]:
history = model.fit(train_img_gen, epochs=100, validation_data=val_img_gen, callbacks=cp)

Epoch 1/100
400/400 [==============================] - ETA: 0s - loss: 0.9106 - accuracy: 0.7867 
Epoch 00001: val_accuracy improved from -inf to 0.74264, saving model to /content/drive/MyDrive/Newcastle University/Machine Learning/Models/resnet50_aug.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


400/400 [==============================] - 8575s 21s/step - loss: 0.9106 - accuracy: 0.7867 - val_loss: 0.9539 - val_accuracy: 0.7426
Epoch 2/100
400/400 [==============================] - ETA: 0s - loss: 0.4874 - accuracy: 0.8277
Epoch 00002: val_accuracy improved from 0.74264 to 0.75212, saving model to /content/drive/MyDrive/Newcastle University/Machine Learning/Models/resnet50_aug.h5
400/400 [==============================] - 243s 608ms/step - loss: 0.4874 - accuracy: 0.8277 - val_loss: 0.7529 - val_accuracy: 0.7521
Epoch 3/100
400/400 [==============================] - ETA: 0s - loss: 0.3960 - accuracy: 0.8612
Epoch 00003: val_accuracy improved from 0.75212 to 0.76509, saving model to /content/drive/MyDrive/Newcastle University/Machine Learning/Models/resnet50_aug.h5
400/400 [==============================] - 243s 608ms/step - loss: 0.3960 - accuracy: 0.8612 - val_loss: 0.7264 - val_accuracy: 0.7651
Epoch 4/100
400/400 [==============================] - ETA: 0s - loss: 0.3119 - ac